# Analyzing FEMA's National Flood Insurance Program (NFIP) Policies Data With DuckDB
Author: Mark Bauer

# OpenFEMA Dataset: FIMA NFIP Redacted Policies - v2
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: FIMA NFIP Redacted Policies - v2. Retrieved from https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about OpenFEMA's  [Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

**Dataset Description**:
>This dataset provides details on NFIP policy transactions. It is derived from the NFIP system of record, staged in the NFIP reporting platform and redacted to protect policy holder personally identifiable information.
>
>This dataset is not intended to be an official federal report, and should not be considered an official federal report.

**About the National Flood Insurance Program**:   
>Congress passed the National Flood Insurance Act (NFIA), 42 U.S.C. 4001 in 1968, creating the National Flood Insurance Program (NFIP) in order to reduce future flood losses through flood hazard identification, floodplain management, and providing insurance protection. The Department of Housing and Urban Development (HUD) originally administered the NFIP, and Congress subsequently transferred the NFIP to FEMA upon its creation in 1979. FEMA and insurance companies participating in FEMA's Write Your Own (WYO) program offer NFIP insurance coverage for building structures as well as for contents and personal property within the building structures to eligible and insurable properties. The WYO program began in 1983 with NFIP operating under Part B of the NFIA and allows FEMA to authorize private insurance companies to issue the Standard Flood Insurance Policy (SFIP) as FEMA's fiduciary and fiscal agent. FEMA administers NFIP by ensuring insurance applications are processed properly; determining correct premiums; renewing, reforming, and cancelling insurance policies; transferring policies from the seller of the property to the purchaser of the property in certain circumstances; and processing insurance claims.
>
>The paid premiums of SFIPs and claims payments for damaged property are processed through the National Flood Insurance Fund (NFIF). NFIF was established by the National Flood Insurance Act of 1968 (42 U.S.C. 4001, et seq.), and is a centralized premium revenue and fee-generated fund that supports NFIP, which holds these U.S. Treasury funds.
>
>The Flood Insurance Claims Manual (https://nfipservices.floodsmart.gov/insurance-manuals) provides claims guidance to WYOs, vendors, adjusters, and examiners so that policyholders experience consistent and reliable service. The Manual provides processes for handling claims from the notice of loss to final payment. The NFIP has provided answers to Frequently Asked Questions (FAQs) to assist the public in understanding and navigating the data our program makes available: https://www.fema.gov/sites/default/files/documents/fema_nfip-data-faqs.pdf.

**Data Dictionary**:  
View the data dictionary on OpenFEMA under the [Data Fields](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2) section.

In [1]:
# import libraries
import duckdb
import pandas as pd

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb,pandas

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0
pandas: 1.5.1



In [3]:
# list datasets
%ls data/

FimaNfipClaims.parquet       claims.db
FimaNfipPolicies.parquet     policies-nyc-year.parquet
claims-nyc-year.parquet      policies-state-year.parquet
claims-state-year.parquet    policies.db


In [4]:
# preview size of file
!du -sh data/* | sort -rh

 11G	data/policies.db
3.2G	data/FimaNfipPolicies.parquet
609M	data/claims.db
194M	data/FimaNfipClaims.parquet
8.0K	data/policies-state-year.parquet
4.0K	data/policies-nyc-year.parquet
4.0K	data/claims-state-year.parquet
4.0K	data/claims-nyc-year.parquet


# Redacted Policies: Before Getting Started
Please note that this dataset provides details on NFIP policies and is ***redacted*** to protect policy holder personally identifiable information. The policie's `latitude` and `longitude` field **should not be used to represent the precise location of the insured building**. From the data dictionary:

>Latitude: **Approximate latitude of the insured building (to 1 decimal place)**. This represents the approximate location of the insured property. The precision has been lessened to ensure individual privacy. This may result in a point location that exists in an incorrect county or state. Use the state and county fields for record aggregation for these dimensions.
>
>Longitude: **Approximate longitude of the insured building (to 1 decimal place)**. This represents the approximate location of the insured property. The precision has been lessened to ensure individual privacy. This may result in a point location that exists in an incorrect county or state. Use the state and county fields for record aggregation for these dimensions.

For more information, visit the [Data Dictionary](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2) and review [Frequently Asked Questions about NFIP Policies and Claims Data](https://nfipservices.floodsmart.gov/frequently-asked-questions-about-nfip-policies-and-claims-data).

This analysis presents all financial figures in nominal dollars. No adjustments for inflation have been made, meaning that the values reflect the actual dollar amounts at the time of measurement. Additionally, the date and time of data access can be found in the [download-data](https://github.com/mebauer/duckdb-fema-nfip/blob/main/download-data.ipynb) notebook.

# Create a DuckDB database instance using the Python client

In [5]:
%%time

# create a DuckDB database instance
con = duckdb.connect("data/policies.db")

# create table claims of dataset
con.execute("""
    CREATE OR REPLACE TABLE policies AS
        FROM read_parquet('data/FimaNfipPolicies.parquet')
""")

# sanity check
con.sql("""
    SELECT *
    FROM policies
    LIMIT 5
""").show()

┌──────────────────────┬────────────────────┬───┬──────────────┬──────────────┬──────────────────────┐
│ agricultureStructu…  │ baseFloodElevation │ … │   latitude   │  longitude   │          id          │
│       boolean        │    decimal(8,2)    │   │ decimal(9,1) │ decimal(9,1) │         uuid         │
├──────────────────────┼────────────────────┼───┼──────────────┼──────────────┼──────────────────────┤
│ false                │               NULL │ … │         32.2 │       -110.9 │ c3c498e0-39ee-4642…  │
│ false                │               NULL │ … │         37.6 │       -121.0 │ 6daee4b7-308b-453c…  │
│ false                │               NULL │ … │         35.4 │       -118.9 │ d4191676-0f6d-47bf…  │
│ false                │               NULL │ … │         39.0 │       -121.4 │ 9dac717a-9a1f-4323…  │
│ false                │               NULL │ … │         34.4 │       -119.6 │ e11197ee-65ef-4630…  │
├──────────────────────┴────────────────────┴───┴──────────────┴─────────

In [6]:
# list tables and schemas
con.sql("SHOW ALL TABLES").df()

,database,schema,name,column_names,column_types,temporary
0,policies,main,policies,"[agricultureStructureIndicator, baseFloodEleva...","[BOOLEAN, DECIMAL(8,2), SMALLINT, DATE, VARCHA...",False


In [7]:
# count of rows
con.sql("""
    SELECT
        COUNT(*) AS count_rows
    FROM policies
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│   69489458 │
└────────────┘

In [8]:
# count of columns
con.sql("""
    SELECT
        COUNT(column_name) AS count_columns
    FROM (DESCRIBE FROM policies)
""")

┌───────────────┐
│ count_columns │
│     int64     │
├───────────────┤
│            81 │
└───────────────┘

# Examine Dataset

## Column Info

In [9]:
# examine column datatypes
con.sql("""
    SELECT
        column_name,
        column_type
    FROM (DESCRIBE policies)
    ORDER BY column_name ASC
""").show(max_rows=80)

┌────────────────────────────────────────┬──────────────┐
│              column_name               │ column_type  │
│                varchar                 │   varchar    │
├────────────────────────────────────────┼──────────────┤
│ AdditionalContentsRate                 │ DECIMAL(8,2) │
│ additionalBuildingRate                 │ DECIMAL(8,2) │
│ agricultureStructureIndicator          │ BOOLEAN      │
│ baseFloodElevation                     │ DECIMAL(8,2) │
│ basementEnclosureCrawlspaceType        │ SMALLINT     │
│ basicBuildingRate                      │ DECIMAL(8,2) │
│ basicContentsRate                      │ DECIMAL(8,2) │
│ buildingDeductibleCode                 │ VARCHAR      │
│ buildingDescriptionCode                │ SMALLINT     │
│ buildingReplacementCost                │ BIGINT       │
│ cancellationDateOfFloodPolicy          │ DATE         │
│ cancellationVoidanceReasonCode         │ VARCHAR      │
│ censusBlockGroupFips                   │ VARCHAR      │
│ censusTract 

In [10]:
# approximate column null percentage
con.sql("""
    SELECT
        column_name,
        null_percentage
    FROM (SUMMARIZE FROM policies)
    ORDER BY null_percentage DESC
""").show(max_rows=80)

┌────────────────────────────────────────┬─────────────────┐
│              column_name               │ null_percentage │
│                varchar                 │  decimal(9,2)   │
├────────────────────────────────────────┼─────────────────┤
│ cancellationDateOfFloodPolicy          │           95.70 │
│ cancellationVoidanceReasonCode         │           95.70 │
│ obstructionType                        │           81.43 │
│ elevationCertificateIndicator          │           77.96 │
│ enclosureTypeCode                      │           74.92 │
│ insuranceToValueCode                   │           69.99 │
│ lowestAdjacentGrade                    │           68.84 │
│ lowestFloorElevation                   │           67.10 │
│ baseFloodElevation                     │           66.54 │
│ elevationDifference                    │           66.12 │
│ propertyPurchaseDate                   │           59.77 │
│ basementEnclosureCrawlspaceType        │           59.02 │
│ subsidizedRateType    

## Preview Data
We want to preview values in each column.

In [11]:
# preview data as pandas dataframe for readability
sql = """
    SELECT *
    FROM policies
    LIMIT 5
"""

# examine each column in sections because of large number of columns
con.sql(sql).df().iloc[:, :15]

,agricultureStructureIndicator,baseFloodElevation,basementEnclosureCrawlspaceType,cancellationDateOfFloodPolicy,condominiumCoverageTypeCode,construction,crsClassCode,buildingDeductibleCode,contentsDeductibleCode,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,federalPolicyFee,ratedFloodZone,hfiaaSurcharge
0,False,NaN,0,NaT,N,False,NaN,1,None,False,None,NaN,35,AE,0
1,False,NaN,0,NaT,N,False,NaN,5,None,False,None,NaN,35,AE,0
2,False,NaN,0,NaT,N,False,NaN,3,None,False,None,NaN,35,A11,0
3,False,NaN,0,NaT,N,False,NaN,0,0,True,None,NaN,13,A,0
4,False,NaN,0,NaT,N,False,NaN,1,0,True,None,NaN,35,AE,0


In [12]:
# slice through columns
con.sql(sql).df().iloc[:, 15:30]

,houseOfWorshipIndicator,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,nonProfitIndicator,numberOfFloorsInInsuredBuilding,obstructionType,occupancyType,originalConstructionDate,originalNBDate,policyCost,policyCount,policyEffectiveDate,policyTerminationDate,policyTermIndicator
0,False,NaN,NaN,NaN,False,1,NaN,1,1973-01-01,2006-04-26,541,1,2009-04-26,2010-04-26,1
1,False,NaN,NaN,NaN,False,1,NaN,1,2005-03-22,2022-01-05,515,1,2009-04-27,2010-04-27,1
2,False,NaN,NaN,NaN,False,1,NaN,1,1986-09-28,2005-05-24,1252,1,2009-05-24,2010-05-24,1
3,False,3.0,NaN,NaN,False,1,10.0,1,1992-07-01,2003-08-31,348,1,2009-08-31,2010-08-31,1
4,False,NaN,NaN,NaN,False,1,10.0,1,1960-01-01,2001-08-05,1251,1,2009-08-05,2010-08-05,1


In [13]:
# slice through columns
con.sql(sql).df().iloc[:, 30:45]

,postFIRMConstructionIndicator,primaryResidenceIndicator,rateMethod,regularEmergencyProgramIndicator,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,totalInsurancePremiumOfThePolicy,cancellationVoidanceReasonCode,subsidizedRateType,iccPremium,reserveFundAssessment,communityProbationSurcharge,premiumPaymentIndicator,buildingReplacementCost
0,False,True,1,R,False,83000,0,506,None,None,75,0,0,NaN,83000
1,True,False,1,R,False,165000,0,480,None,None,6,0,0,NaN,165000
2,False,True,1,R,False,250000,0,1217,None,None,60,0,0,NaN,250000
3,True,False,7,R,False,250000,100000,335,None,None,6,0,0,NaN,0
4,False,True,1,R,False,250000,0,1216,None,None,60,0,0,NaN,0


In [14]:
# slice through columns
con.sql(sql).df().iloc[:, 45:60]

,basicBuildingRate,additionalBuildingRate,basicContentsRate,AdditionalContentsRate,enclosureTypeCode,buildingDescriptionCode,insuranceToValueCode,postFirmVzoneIndicator,floodproofedIndicator,waitingPeriodType,rolloverTransferCode,endorsementEffectiveDate,propertyPurchaseDate,rentalPropertyIndicator,tenantIndicator
0,0.76,0.54,0.0,0.0,None,1,NaN,False,False,None,N,2009-04-26,NaT,False,False
1,0.78,0.21,0.0,0.0,N,1,NaN,False,False,None,N,2009-04-27,NaT,False,False
2,0.76,0.54,0.0,0.0,None,1,NaN,False,False,None,N,2009-05-24,NaT,False,False
3,0.00,0.00,0.0,0.0,None,1,NaN,False,False,None,R,2009-08-31,NaT,False,False
4,0.76,0.54,0.0,0.0,None,1,NaN,False,False,None,R,2009-08-05,NaT,False,False


In [15]:
# slice through columns
con.sql(sql).df().iloc[:, 60:75]

,stateOwnedIndicator,disasterAssistanceCoverageRequiredCode,mandatoryPurchaseFlag,grandfatheringTypeCode,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,programTypeIndicator,mapPanelNumber,mapPanelSuffix,floodZoneCurrent,femaRegion,propertyState,reportedCity,reportedZipCode
0,False,0,True,1,040076,0000,None,False,2282,L,AE,9,AZ,Currently Unavailable,85712
1,False,0,False,1,060384,0000,None,False,0532,F,None,9,CA,Currently Unavailable,95351
2,False,0,True,1,060075,0000,None,False,0745,B,None,9,CA,Currently Unavailable,93308
3,False,0,False,1,060460,0000,None,False,0045,D,None,9,CA,Currently Unavailable,95692
4,False,0,True,1,060331,0000,None,False,1392,H,AE,9,CA,Currently Unavailable,93108


In [16]:
# slice through columns
con.sql(sql).df().iloc[:, 75:]

,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,04019,04019003101,040190031011,32.2,-110.9,c3c498e0-39ee-4642-9537-bfd386347a70
1,06099,06099001603,060990016031,37.6,-121.0,6daee4b7-308b-453c-a1c3-6eab8dd90ab0
2,06029,06029005103,060290051031,35.4,-118.9,d4191676-0f6d-47bf-850c-08836f79cb58
3,06115,06115040800,061150408001,39.0,-121.4,9dac717a-9a1f-4323-8bb4-02e327e7a2ca
4,06083,06083001402,060830014022,34.4,-119.6,e11197ee-65ef-4630-a588-771637842dc8


In [17]:
# count duplicate IDs
con.sql("""
    SELECT
        id,
        COUNT(id) AS count
    FROM policies
    GROUP BY id
    HAVING count > 1
""").show()

┌──────┬───────┐
│  id  │ count │
│ uuid │ int64 │
├──────┴───────┤
│    0 rows    │
└──────────────┘



In [18]:
# earliest and latest record effective date
con.sql("""
    SELECT
        min(policyEffectiveDate) AS earliestPolicyEffectiveDate,
        max(policyEffectiveDate) AS earliestPolicyEffectiveDate
    FROM policies
""").show()

┌─────────────────────────────┬─────────────────────────────┐
│ earliestPolicyEffectiveDate │ earliestPolicyEffectiveDate │
│            date             │            date             │
├─────────────────────────────┼─────────────────────────────┤
│ 2009-01-01                  │ 2025-12-12                  │
└─────────────────────────────┴─────────────────────────────┘



In [19]:
# total insured units in dataset
con.sql("""
    SELECT
        SUM(policyCount) AS totalPolicyCount
    FROM policies
""").show()

┌──────────────────┐
│ totalPolicyCount │
│      int128      │
├──────────────────┤
│         85679492 │
└──────────────────┘



Policy Count:
>Insured units in an active status. A policy contract ceases to be in an active status as of the cancellation date or the expiration date. Residential Condominium Building Association Policy (RCBAP) contracts are stored as a single policy contract but insure multiple units and therefore represent multiple policies.

Source: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2

# Analysis
`Policies in force` is the number of policies as of the date listed on the OpenFEMA NFIP Redacted Policies dataset page.

In this notebook, it's **2025-05-14**. We'll mostly focus on this statistic in this analysis.

Note: All dollar amounts are reported in nominal terms and have not been adjusted for inflation.

**Table xx: Policies in Force Totals**

In [20]:
con.sql("""
    SELECT
        SUM(policyCount)::BIGINT AS policiesInForce,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalInsuranceCoverageM,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 2) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 3) AS policyCostM
    FROM policies
    WHERE policyTerminationDate > '2025-05-14'
""").df()

,policiesInForce,totalInsuranceCoverageM,totalInsurancePremiumOfThePolicyM,policyCostM
0,4469069,1229228.66,3874.69,5047.91


**Table xx: Top 20 Counties by Policies in Force**

In [21]:
con.sql("""
    SELECT
        propertyState,
        countyCode,
        SUM(policyCount)::BIGINT AS policiesInForce,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalInsuranceCoverageM,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 2) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 3) AS policyCostM,
    FROM policies
    WHERE policyTerminationDate > '2025-05-14'
    GROUP BY ALL
    ORDER BY policiesInForce DESC
    LIMIT 20
""").df()

,propertyState,countyCode,policiesInForce,totalInsuranceCoverageM,totalInsurancePremiumOfThePolicyM,policyCostM
0,FL,12086,330846,82859.27,187.25,245.272
1,TX,48201,250215,78227.44,211.96,275.826
2,FL,12011,222488,58256.18,118.14,157.886
3,FL,12103,131941,32630.99,169.46,207.940
4,FL,12071,130839,33802.48,178.42,223.405
5,FL,12099,124284,34566.24,62.06,83.932
6,FL,12021,97796,25821.42,103.80,132.312
7,LA,22051,84618,24435.86,77.95,101.979
8,LA,22071,66715,19767.68,54.34,73.629
9,FL,12057,65428,18495.84,82.36,102.336


**Table xx: Top Occupancy Types by Policies in Force**

In [22]:
con.sql("""
    SELECT
        occupancyType,
        SUM(policyCount)::BIGINT AS policiesInForce,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 2) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 2) AS policyCostM,
        ROUND(SUM(totalBuildingInsuranceCoverage) / 1_000_000.0, 2) AS totalBuildingInsuranceCoverageM,
        ROUND(SUM(totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalContentsInsuranceCoverageM,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalInsuranceCoverageM
    FROM policies 
    WHERE policyTerminationDate > '2025-05-14'
    GROUP BY ALL
    ORDER BY policiesInForce DESC   
""").df()

,occupancyType,policiesInForce,totalInsurancePremiumOfThePolicyM,policyCostM,totalBuildingInsuranceCoverageM,totalContentsInsuranceCoverageM,totalInsuranceCoverageM
0,11,2840286,2582.85,3357.35,638605.70,188292.93,826898.64
1,15,1079861,421.64,533.61,222420.73,412.69,222833.42
2,18,225970,553.31,720.02,73612.73,30860.96,104473.70
3,16,119393,59.45,88.02,14034.84,6239.19,20274.03
4,12,93062,96.69,135.96,20120.16,3456.94,23577.10
5,14,53180,45.75,60.22,5126.88,1162.10,6288.98
6,13,52106,108.88,143.96,22290.53,1424.12,23714.65
7,19,4523,5.46,7.78,0.00,1088.42,1088.42
8,17,688,0.68,1.00,64.79,14.94,79.73


**Table xx: Top 20 State and Occupancy Types by Policies in Force**

In [23]:
con.sql("""
    SELECT
        propertyState,
        occupancyType,
        SUM(policyCount)::BIGINT AS policiesInForce,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 2) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 2) AS policyCostM,
        ROUND(SUM(totalBuildingInsuranceCoverage) / 1_000_000.0, 2) AS totalBuildingInsuranceCoverageM,
        ROUND(SUM(totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalContentsInsuranceCoverageM,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalInsuranceCoverageM
    FROM policies 
    WHERE policyTerminationDate > '2025-05-14'
    GROUP BY ALL
    ORDER BY policiesInForce DESC  
    LIMIT 20
""").df()

,propertyState,occupancyType,policiesInForce,totalInsurancePremiumOfThePolicyM,policyCostM,totalBuildingInsuranceCoverageM,totalContentsInsuranceCoverageM,totalInsuranceCoverageM
0,FL,11,880195,906.51,1162.43,207363.08,59069.21,266432.28
1,FL,15,641607,269.30,333.57,133199.68,226.89,133426.57
2,TX,11,516879,407.85,531.71,116464.24,41183.59,157647.83
3,LA,11,325176,271.22,351.13,69337.54,21679.15,91016.68
4,CA,11,127984,108.63,142.65,30383.64,8397.83,38781.47
5,SC,11,113555,89.13,118.47,27206.61,8926.70,36133.30
6,NY,11,99969,114.59,146.51,22632.43,6134.03,28766.46
7,NJ,11,98009,103.50,137.38,22627.41,4916.79,27544.20
8,NC,11,97026,74.69,102.40,22134.99,6279.94,28414.93
9,NJ,15,72760,32.19,43.25,15995.60,24.82,16020.42


**Table xx: Top 20 Flood Zones by Policies in Force**

In [24]:
con.sql("""
    SELECT
        ratedFloodZone,
        SUM(policyCount)::BIGINT AS policiesInForce,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 2) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 2) AS policyCostM,
        ROUND(SUM(totalBuildingInsuranceCoverage) / 1_000_000.0, 2) AS totalBuildingInsuranceCoverageM,
        ROUND(SUM(totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalContentsInsuranceCoverageM,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalInsuranceCoverageM
    FROM policies 
    WHERE policyTerminationDate > '2025-05-14'
    GROUP BY ALL
    ORDER BY policiesInForce DESC  
    LIMIT 20    
""").df()

,ratedFloodZone,policiesInForce,totalInsurancePremiumOfThePolicyM,policyCostM,totalBuildingInsuranceCoverageM,totalContentsInsuranceCoverageM,totalInsuranceCoverageM
0,AE,2060763,2158.13,2777.91,453456.44,73398.30,526854.74
1,X,1685438,1091.78,1467.10,387316.99,131876.04,519193.02
2,AH,172758,112.20,151.97,38877.06,7208.07,46085.13
3,A,169715,196.68,237.07,30172.57,5501.41,35673.98
4,VE,150783,138.14,177.71,34186.64,3236.06,37422.70
5,AO,70721,52.01,69.85,16348.99,1628.67,17977.66
6,A99,37129,23.39,32.00,9223.94,2910.39,12134.33
7,C,36689,26.08,34.60,8450.71,3231.57,11682.27
8,B,21837,16.07,21.39,4892.83,1741.28,6634.10
9,AHB,11109,5.09,7.14,2416.43,478.97,2895.40


**Table xx: Top 20 States and Flood Zones by Policies in Force**

In [25]:
con.sql("""
    SELECT
        propertyState,
        ratedFloodZone,
        SUM(policyCount)::BIGINT AS policiesInForce,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 2) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 2) AS policyCostM,
        ROUND(SUM(totalBuildingInsuranceCoverage) / 1_000_000.0, 2) AS totalBuildingInsuranceCoverageM,
        ROUND(SUM(totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalContentsInsuranceCoverageM,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalInsuranceCoverageM
    FROM policies 
    WHERE policyTerminationDate > '2025-05-14'
    GROUP BY ALL
    ORDER BY policiesInForce DESC  
    LIMIT 20    
""").df()

,propertyState,ratedFloodZone,policiesInForce,totalInsurancePremiumOfThePolicyM,policyCostM,totalBuildingInsuranceCoverageM,totalContentsInsuranceCoverageM,totalInsuranceCoverageM
0,FL,AE,867284,901.87,1143.48,192637.07,26449.46,219086.53
1,FL,X,519984,296.35,400.18,118847.01,35643.48,154490.48
2,TX,X,382673,273.49,360.50,88839.85,34867.23,123707.07
3,LA,X,186493,136.38,182.92,43093.85,16208.91,59302.76
4,TX,AE,161005,176.72,229.60,35097.75,8881.55,43979.30
5,LA,AE,150671,157.57,204.55,30631.34,6904.06,37535.40
6,FL,AH,147569,90.32,122.51,33515.67,6382.21,39897.87
7,NJ,AE,139418,143.55,188.91,32183.41,4056.63,36240.04
8,SC,AE,97971,77.35,101.79,22104.44,4199.63,26304.07
9,NY,AE,85369,118.30,149.43,20265.30,3255.86,23521.16


**Table xx: Policies by Effective Year**

In [26]:
df = con.sql("""
    SELECT
        strftime('%Y', CAST(policyEffectiveDate AS TIMESTAMP)) AS policyEffectiveYear,
        SUM(policyCount)::BIGINT AS policiesInForce,
        ROUND(SUM(totalInsurancePremiumOfThePolicy) / 1_000_000.0, 2) AS totalInsurancePremiumOfThePolicyM,
        ROUND(SUM(policyCost) / 1_000_000.0, 2) AS policyCostM,
        ROUND(SUM(totalBuildingInsuranceCoverage) / 1_000_000.0, 2) AS totalBuildingInsuranceCoverageM,
        ROUND(SUM(totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalContentsInsuranceCoverageM,
        ROUND(SUM(totalBuildingInsuranceCoverage + totalContentsInsuranceCoverage) / 1_000_000.0, 2) AS totalInsuranceCoverageM
    FROM policies 
    GROUP BY ALL
    ORDER BY policyEffectiveYear ASC      
""").df()

df

,policyEffectiveYear,policiesInForce,totalInsurancePremiumOfThePolicyM,policyCostM,totalBuildingInsuranceCoverageM,totalContentsInsuranceCoverageM,totalInsuranceCoverageM
0,2009,5712917,3132.08,3274.89,1014314.81,240328.06,1254642.87
1,2010,5661703,3283.30,3445.79,1027562.34,240697.39,1268259.72
2,2011,5680679,3407.19,3577.04,1050676.29,249675.30,1300351.59
3,2012,5641407,3504.03,3672.47,1059369.02,252524.57,1311893.59
4,2013,5647055,3724.66,3927.77,1074863.86,259070.84,1333934.70
5,2014,5525767,3713.83,4040.59,1078007.56,259538.86,1337546.42
6,2015,5257480,3473.53,4441.18,1042901.09,250213.71,1293114.80
7,2016,5233261,3512.31,4673.95,1053072.61,256959.77,1310032.38
8,2017,5205874,3467.77,4601.70,1066041.69,268312.21,1334353.90
9,2018,5187350,3462.80,4571.60,1077662.41,275877.69,1353540.00


# Sample Workflow of Saving Files

## Saving Policies by State and Effective Year

In [27]:
# sanity check with pandas
df = con.sql("""
    SELECT
        UPPER(propertyState) AS propertyState,
        strftime('%Y', CAST(policyEffectiveDate AS TIMESTAMP)) AS policyEffectiveYear,
        SUM(policyCount)::BIGINT AS policiesInForce
    FROM policies 
    GROUP BY ALL
    ORDER BY propertyState ASC, policyEffectiveYear ASC      
""").df()

df

,propertyState,policyEffectiveYear,policiesInForce
0,AK,2009,2926
1,AK,2010,2943
2,AK,2011,3068
3,AK,2012,3148
4,AK,2013,3245
...,...,...,...
947,WY,2021,1668
948,WY,2022,1689
949,WY,2023,1741
950,WY,2024,1689


In [28]:
# write out as Parquet file
con.sql("""
    COPY (
        SELECT
            UPPER(propertyState) AS propertyState,
            strftime('%Y', CAST(policyEffectiveDate AS TIMESTAMP)) AS policyEffectiveYear,
            SUM(policyCount)::BIGINT AS policiesInForce
        FROM policies 
        GROUP BY ALL
        ORDER BY propertyState ASC, policyEffectiveYear ASC
    )
    TO 'data/policies-state-year.parquet' (FORMAT 'parquet');
""")

%ls data/

FimaNfipClaims.parquet       claims.db
FimaNfipPolicies.parquet     policies-nyc-year.parquet
claims-nyc-year.parquet      policies-state-year.parquet
claims-state-year.parquet    policies.db


In [29]:
# sanity check
duckdb.sql("""
    SELECT *
    FROM 'data/policies-state-year.parquet'
    LIMIT 10
""")

┌───────────────┬─────────────────────┬─────────────────┐
│ propertyState │ policyEffectiveYear │ policiesInForce │
│    varchar    │       varchar       │      int64      │
├───────────────┼─────────────────────┼─────────────────┤
│ AK            │ 2009                │            2926 │
│ AK            │ 2010                │            2943 │
│ AK            │ 2011                │            3068 │
│ AK            │ 2012                │            3148 │
│ AK            │ 2013                │            3245 │
│ AK            │ 2014                │            3144 │
│ AK            │ 2015                │            2903 │
│ AK            │ 2016                │            2751 │
│ AK            │ 2017                │            2569 │
│ AK            │ 2018                │            2478 │
├───────────────┴─────────────────────┴─────────────────┤
│ 10 rows                                     3 columns │
└───────────────────────────────────────────────────────┘

In [30]:
# sanity check
duckdb.sql("""
    SELECT COUNT(*) AS count
    FROM 'data/policies-state-year.parquet'
""")

┌───────┐
│ count │
│ int64 │
├───────┤
│   952 │
└───────┘

## Saving Policies by New York City Counties (Boroughs) and Effective Year

In [31]:
con.sql("""
    COPY (
        SELECT
            UPPER(propertyState) AS propertyState,
            countyCode,
            strftime('%Y', CAST(policyEffectiveDate AS TIMESTAMP)) AS policyEffectiveYear,
            SUM(policyCount)::BIGINT AS policiesInForce
        FROM policies 
        WHERE countyCode IN ('36005', '36047', '36061', '36081', '36085')
        GROUP BY ALL
        ORDER BY countyCode ASC, policyEffectiveYear ASC
    )
    TO 'data/policies-nyc-year.parquet' (FORMAT 'parquet');
""")

%ls data/

FimaNfipClaims.parquet       claims.db
FimaNfipPolicies.parquet     policies-nyc-year.parquet
claims-nyc-year.parquet      policies-state-year.parquet
claims-state-year.parquet    policies.db


In [32]:
# sanity check
duckdb.sql("""
    SELECT *
    FROM 'data/policies-nyc-year.parquet'
    LIMIT 10
""")

┌───────────────┬────────────┬─────────────────────┬─────────────────┐
│ propertyState │ countyCode │ policyEffectiveYear │ policiesInForce │
│    varchar    │  varchar   │       varchar       │      int64      │
├───────────────┼────────────┼─────────────────────┼─────────────────┤
│ NY            │ 36005      │ 2009                │            3129 │
│ NY            │ 36005      │ 2010                │            3168 │
│ NY            │ 36005      │ 2011                │            3145 │
│ NY            │ 36005      │ 2012                │            3194 │
│ NY            │ 36005      │ 2013                │            3252 │
│ NY            │ 36005      │ 2014                │            3255 │
│ NY            │ 36005      │ 2015                │            3164 │
│ NY            │ 36005      │ 2016                │            3159 │
│ NY            │ 36005      │ 2017                │            3119 │
│ NY            │ 36005      │ 2018                │            3086 │
├─────

In [33]:
# sanity check
duckdb.sql("""
    SELECT COUNT(*) AS count
    FROM 'data/policies-nyc-year.parquet'
""")

┌───────┐
│ count │
│ int64 │
├───────┤
│   161 │
└───────┘

In [34]:
# preview size of file
!du -sh data/* | sort -rh

5.5G	data/policies.db
3.2G	data/FimaNfipPolicies.parquet
609M	data/claims.db
194M	data/FimaNfipClaims.parquet
8.0K	data/policies-state-year.parquet
4.0K	data/policies-nyc-year.parquet
4.0K	data/claims-state-year.parquet
4.0K	data/claims-nyc-year.parquet


In [35]:
# close connection
con.close()